In [7]:
import pandas as pd
import numpy as np
import datetime as dt
import os
import time
import xgboost as xgb
from sklearn.metrics import mean_squared_error, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

pd.set_option('mode.chained_assignment', None)

In [17]:
#constants

data_file = 'processed_data.csv'

In [18]:
data = pd.read_csv(data_file, usecols=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])

In [19]:
data.groupby(by='class').count()

,v0,v1,v2,a0,a1,a2,max(a),min(a),avg(a),var(a),max(v),min(v),avg(v),var(v)
class,,,,,,,,,,,,,,
bike,79307,79307,79307,79307,79307,79307,79307,79307,79307,79307,79307,79307,79307,79307
bus,30903,30903,30903,30903,30903,30903,30903,30903,30903,30903,30903,30903,30903,30903
car,214843,214843,214843,214843,214843,214843,214843,214843,214843,214843,214843,214843,214843,214843
train,12571,12571,12571,12571,12571,12571,12571,12571,12571,12571,12571,12571,12571,12571
walk,145433,145433,145433,145433,145433,145433,145433,145433,145433,145433,145433,145433,145433,145433


In [20]:
data['class'].unique()

array(['car', 'walk', 'bike', 'bus', 'train'], dtype=object)

In [12]:
data['class'][data['class'] != 'car'] = 'not-car'

In [21]:
X, y = data.iloc[:,:-1],data.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [22]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 1, learning_rate = 0.5,
                max_depth = 10, n_estimators = 10)

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bytree=1, gamma=0, learning_rate=0.5, max_delta_step=0,
              max_depth=10, min_child_weight=1, missing=None, n_estimators=10,
              n_jobs=1, nthread=None, objective='multi:softprob',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=True, subsample=1)

In [23]:
from sklearn.metrics import classification_report
preds = xg_reg.predict(X_test)
print(classification_report(y_test,preds))
print(confusion_matrix(y_test,preds), labels=['car', 'walk', 'bike', 'bus', 'train'])

              precision    recall  f1-score   support

        bike       0.71      0.64      0.67     16004
         bus       0.49      0.07      0.13      6076
         car       0.79      0.72      0.76     42812
       train       0.75      0.43      0.55      2526
        walk       0.63      0.88      0.74     29194

    accuracy                           0.71     96612
   macro avg       0.68      0.55      0.57     96612
weighted avg       0.71      0.71      0.69     96612



In [ ]:
import matplotlib.pyplot as plt
xgb.plot_importance(xg_reg)
plt.rcParams['figure.figsize'] = [5, 5]
plt.show()